In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_train_preprocesado.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_test_preprocesado.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

In [ ]:
x_train_text = df_train['review_es']
y_train_text = df_train['sentimiento']

x_test_text = df_test['review_es']

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score

## Vectorizar

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
my_stop_words = stopwords.words('spanish')

# Instanciamos el CV
#Configurar el vectorizador de palabras para convertir el texto en una representación numérica.
vec = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2))

# Ajustamos el CV y transformamos los datos
#transformar los datos de entrenamiento en una matriz numérica.
x_train_aux = vec.fit_transform(x_train_text)
#Transformar los datos de prueba en la misma representación numérica que el conjunto de entrenamiento
x_test = vec.transform(x_test_text)

##n=10, folds=10

In [ ]:
 #Cantidad de combinaciones que quiero porbar
n=10
#Conjunto de parámetros que quiero usar
params_grid = {'criterion':["entropy", "gini"],
               'min_samples_leaf':list(range(3,15)),
               'min_samples_split':list(range(3,20)),
               'n_estimators':list(range(25,60)),
               }
#Cantidad de splits para el Cross Validation
folds=10
#Kfold estratificado
kfoldcv = KFold(n_splits=folds)
#Clasificador
base_tree = RandomForestClassifier(oob_score=True, random_state=1, n_jobs=-1)
#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(f1_score, average='micro')
#Random Search Cross Validation
randomcv = RandomizedSearchCV(estimator=base_tree,
                              param_distributions = params_grid,
                              scoring=scorer_fn,
                              cv=kfoldcv,
                              random_state=20,
                                n_iter=n)
#Busco los hiperparamtros que optimizan F1 Score
 randomcv.fit(x_train_aux, y_train_text);

### Prediccion y armado de df

In [ ]:
  #Mejores hiperparametros del arbol
print(randomcv.best_params_)
#Mejor métrica
print(randomcv.best_score_)

{'n_estimators': 57, 'min_samples_split': 14, 'min_samples_leaf': 5, 'criterion': 'entropy'}
0.8419435998473993


In [ ]:
#Mejor estimador
best_rf=randomcv.best_estimator_

#Predicción con mejor estimador
y_pred_rf=best_rf.predict(x_test)

In [ ]:
df_pred_rf = pd.DataFrame()
df_pred_rf['ID'] = df_test['ID']
df_pred_rf['sentimiento'] = y_pred_rf

In [ ]:
df_pred_rf.to_csv('df_pred_rf_n10cv10.csv', index=False)

In [ ]:
import joblib

In [ ]:
joblib.dump(best_rf,'rf_n10cv10.joblib')